In [37]:
# check python version
import sys
print(sys.version)

3.11.5 | packaged by Anaconda, Inc. | (main, Sep 11 2023, 13:26:23) [MSC v.1916 64 bit (AMD64)]


In [9]:
# to test if correctly installed tensorflow
import tensorflow as tf
print(tf.reduce_sum(tf.random.normal([1000, 1000])))

tf.Tensor(293.4349, shape=(), dtype=float32)


In [10]:
#conda install keras

In [11]:
import numpy as np 
import pandas as pd 
#from tensorflow import keras
import keras
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
#from np_utils import to_categorical
import re

In [21]:
# load in data
loadDataInfo = './tmpData/Data/Electrodes/electrode_1.csv'
Data = pd.read_csv(loadDataInfo)
# summarize first few rows
#print(series.head())
print(len(Data))
#refer to the row index:
print(Data.index)
print(Data.columns)
print(Data.shape)

1439
RangeIndex(start=0, stop=1439, step=1)
Index(['-0.641177123205187', '-0.870347381069106', '-0.292730969159417',
       '-0.183297227557644', '-0.418725436420376', '-0.449430723873507',
       '0.956021499890071', '0.947802050191783', '-0.164164421793905',
       '-0.688891818533208',
       ...
       '-0.0088742394076997', '-0.00773404731753397', '-0.0065593974725564',
       '-0.00537458416411922', '-0.00419775527310631', '-0.00305221869486746',
       '-0.00195430940973534', '0.0186712158620674', '0.019213546522994',
       '0.0197054525137061'],
      dtype='object', length=300)
(1439, 300)


In [22]:
data_array = np.array(Data)

n_samples = data_array.shape[0]
n_features = data_array.shape[1]
print([n_samples,n_features])

[1439, 300]


In [23]:
bin_cur = 1
bin_past = 3

bins_predict = bin_cur + bin_past # 3 previous + 1 current

formatted_data = np.zeros([n_samples,bins_predict,n_features])

start_idx=0
for i in range(n_samples-bin_past):
    end_idx=start_idx+bins_predict # indices for 4 elements (sliding fashion)
    formatted_data[i+bin_past,:,:]=data_array[start_idx:end_idx,:] # indexing the data with indices from the previous step
    start_idx=start_idx+1

In [24]:
formatted_data[np.isnan(formatted_data)] = 0 # replacing nans with zeros

In [25]:
# create a label set

n_zones = 45
n_trials = 32

In [33]:
## one-hot-encoding 
cat_zones = keras.utils.to_categorical(np.r_[1:n_zones+1] ,num_classes=46) # one-hot-encoding 
cat_zones = np.delete(cat_zones,0,1)

print(cat_zones)
print(np.shape(cat_zones))
print(type(cat_zones))

[[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]]
(45, 45)
<class 'numpy.ndarray'>


In [35]:
## repeating the labels for 32 (no_of trial) times
labels_all_trials = np.tile(cat_zones, 32) 
print(labels_all_trials)
print(np.shape(labels_all_trials))

[[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]]
(45, 1440)


In [ ]:
## reshaping the data from  (45,1440) to (1440,45)
reshaped_labels_all_trials = np.reshape(labels_all_trials, [n_samples, 45])

In [36]:
all_ind = np.random.choice(np.arange(0, n_samples, 1), n_samples, replace=False)
print(all_ind)

[1279  169 1341 ...  727  513  363]


In [ ]:
# test - validation - train split (20/20/60)
## shuffling numbers from 0 to 1439 to use them as indices 
all_ind = np.random.choice(np.arange(0, n_samples, 1), n_samples, replace=False) # replace=False --> unique  
test_ind = all_ind[:int(n_samples/5)] # test --> 20%
validation_ind = all_ind[int(n_samples/5):int(n_samples/5+n_samples/5)] #  validation --> 20% 
train_ind = all_ind[int(n_samples/5+n_samples/5):] # train --> 60%


In [ ]:
# comparing the elements of test - validation - train split 

comparison_matrix = train_ind[:, np.newaxis] == test_ind
true_ind = np.where(comparison_matrix == True)
repeat = np.shape(true_ind)

if not repeat[1] == 0 :
    raise SystemExit("Repeated elements in TrainTest!")

comparison_matrix2 = train_ind[:, np.newaxis] == validation_ind
true_ind = np.where(comparison_matrix2 == True)
repeat = np.shape(true_ind)

if not repeat[1] == 0 :
       raise SystemExit("Repeated elements in TrainTest!")
    
comparison_matrix3 = test_ind[:, np.newaxis] == validation_ind
true_ind = np.where(comparison_matrix3 == True)
repeat = np.shape(true_ind)

if not repeat[1] == 0 :
    raise SystemExit("Repeated elements in TestValidation!")
          